## Browse and Derived place ontologies

`api.browse()` supports a set of places filters around [category](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/places-category-system-full.html), [cusine](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/food-types-category-system-full.html), [chain](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/places-chain-system-full.html) IDs.

For example, to get the nearby places with on of the category IDs `700-7600-0000` (Fueling Station), `700-7600-0116` (Petrol-Gasoline Station) or `700-7600-0444` (Hydrogen Fuel Station), on can send the following Browse request:

In [1]:
from here_search.api import API

api=API()
resp = await api.browse(latitude=52, longitude=13, limit=4, 
                        categories=["700-7600-0000", "700-7600-0116", "700-7600-0444"])
resp.req.full

'https://browse.search.hereapi.com/v1/browse?at=52%2C13&categories=700-7600-0000%2C700-7600-0116%2C700-7600-0444&limit=4&apiKey=(...)'

In [2]:
resp.titles

['ARAL', 'TOTAL', 'Raiffeisen', 'Shell']

The place filters multi-values allows the support for simple derived ontologies. For example:

| ontology<br/>name | categories | cuisines | chains |
|:-----|:-----------|:---------|:-------|
| gas  | `700-7600-0000,700-7600-0116,700-7600-0444 ` | | |
| eat  | `100`      | | |
| sleep | `500-5000` | | |
| truck stop | `700-7900-0131,700-7900-0132` | | |
| ATM | `700-7010-0108` | | |
| Pizza | | `800-057` | |
| Les Routiers | | | `4406` |


In [3]:
from here_search.entities import OntologyItem, Ontology
from collections import namedtuple

ontology = Ontology([OntologyItem(name, categories, food_types, chains)
#-------------------------------------------------------------------
        for       name,        categories,      food_types,    chains
#-------------------------------------------------------------------
        in     (("gas",     ("700-7600-0000", 
                             "700-7600-0116", 
                             "700-7600-0444"  ), None,         None  ),
                ("eat",     ("100",           ), None,         None  ),
                ("sleep",   ("500-5000",      ), None,         None  ),
                ("stop",    ("700-7900-0131",
                             "700-7900-0132", ), None,         None  ),
                ("ATM",     ("700-7010-0108", ), None,         None  ),
                ("pizza",    None,              ("800-057", ), None  ),
                ("routiers", None,               None,        ("4406", )))])
#--------------------------------------------------------------------
tuple(ontology)

(gas(categories=('700-7600-0000', '700-7600-0116', '700-7600-0444'), foodTypes=None, chains=None),
 eat(categories=('100',), foodTypes=None, chains=None),
 sleep(categories=('500-5000',), foodTypes=None, chains=None),
 stop(categories=('700-7900-0131', '700-7900-0132'), foodTypes=None, chains=None),
 ATM(categories=('700-7010-0108',), foodTypes=None, chains=None),
 pizza(categories=None, foodTypes=('800-057',), chains=None),
 routiers(categories=None, foodTypes=None, chains=('4406',)))

In [4]:
resp = await api.browse(latitude=52, longitude=13, limit=4, **ontology.pizza.mapping)
display(resp.req.full)
resp.titles

'https://browse.search.hereapi.com/v1/browse?at=52%2C13&limit=4&foodTypes=%28%27800-057%27%2C%29&apiKey=(...)'

['Super Pizza', 'Il-Mulino-Due', 'Wasserpfeifchen', 'First Grill']

## Request & Response objects

The `api.get()` method is called by `api.browse()` and the other endpoint methods. It takes a `Request` object and returns a `Response` object.

In [17]:
from here_search.entities import Endpoint, Request, Response
from here_search.api import base_url

req = Request(Endpoint.REVGEOCODE, base_url[Endpoint.REVGEOCODE], {"at": "52,13", "limit": 1})
await api.get(req)

Response(req=Request(endpoint=<Endpoint.REVGEOCODE: 6>, url='https://revgeocode.search.hereapi.com/v1/revgeocode', params={'at': '52,13', 'limit': 1, 'apiKey': '(...)'}, x_headers=None), data={'items': [{'title': 'Niedergörsdorfer Allee, 14913 Niedergörsdorf, Deutschland', 'id': 'here:af:streetsection:.GSuwaNujk32oWouTtr19D', 'resultType': 'street', 'address': {'label': 'Niedergörsdorfer Allee, 14913 Niedergörsdorf, Deutschland', 'countryCode': 'DEU', 'countryName': 'Deutschland', 'stateCode': 'BB', 'state': 'Brandenburg', 'countyCode': 'TF', 'county': 'Teltow-Fläming', 'city': 'Niedergörsdorf', 'street': 'Niedergörsdorfer Allee', 'postalCode': '14913'}, 'position': {'lat': 52.00028, 'lng': 13.00166}, 'distance': 118, 'mapView': {'west': 13.00058, 'south': 51.98972, 'east': 13.00786, 'north': 52.00263}}]}, x_headers={'X-Request-Id': 'REQ-77a849fe-0101-4a5d-a6dc-cd76231a4141', 'X-Correlation-ID': '1a4219b9-8551-4bc0-85fa-223399157797'})